In [1]:
import scrapy
from scrapy.selector import Selector
from scrapy.crawler import CrawlerProcess

import pandas as pd

In [2]:
# định nghĩa lớp kế thừa từ Spider
class giavang_spider(scrapy.Spider):
    name = "vang24h_spider"
    start_urls= ['https://www.24h.com.vn/gia-vang-hom-nay-c425.html',]
    
    custom_settings = {
        'ITEM_PIPELINES':{'__main__.DataCleaning':1},
        'FEED_FORMAT':'csv',
        'FEED_URI':'data/tygiavang.csv'
    }
    
    def parse(self, response):
        banggia = response.css('table.gia-vang-search-data-table')
        ngay = response.css('span.inputDate::text').get()
        for tr in banggia.css('tbody > tr'):
            loai = tr.css('h2::text').get(default='')
            gias = tr.css('span.fixW::text').getall()
            giamua = gias[0]
            giaban = gias[1]
            yield {'ngay': ngay, 'loai' : loai, 'giamua' : giamua, 'giaban': giaban}

In [3]:
# định nghĩa lớp làm sạch DL
class DataCleaning(object):
    def process_item(self, item, spider): 
        ngay = dict(item)['ngay']
        loai = dict(item)['loai']
        giamua = dict(item)['giamua']
        giaban = dict(item)['giaban']
        
        ngay = ngay.replace('\r\n','').strip()
        giamua =(giamua.replace(',',''))
        giaban = float(giaban.replace(',',''))
        
        return {'ngay': ngay, 'loaivang' : loai, 'giamuavao' : giamua, 'giabanra': giaban}

In [4]:
process = CrawlerProcess()
process.crawl(giavang_spider)
process.start()

2023-03-11 17:44:21 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: scrapybot)
2023-03-11 17:44:21 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.12, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 23.0.0 (OpenSSL 3.0.8 7 Feb 2023), cryptography 39.0.2, Platform Windows-10-10.0.22621-SP0
2023-03-11 17:44:21 [scrapy.crawler] INFO: Overridden settings:
{}
2023-03-11 17:44:21 [py.warnings] WARNING: C:\Program Files\Python39\lib\site-packages\scrapy\utils\request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation 